<a href="https://colab.research.google.com/github/Blijf/Datathon/blob/master/Datathon_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CARGA DE DATOS

#### Imports

In [0]:
#INTALL PACKAGES
#!pip install category_encoders
#IMPORTS
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from pandas.plotting import scatter_matrix
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.model_selection import train_test_split
from sklearn.pipeline import FeatureUnion, make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.tree import export_graphviz
from sklearn import preprocessing

import os
from graphviz import Source
import plotly.plotly as py
import cufflinks as cf


In [0]:
def histogramas(train):  
  for col in train:

    #regla de sturges 
    k=int(np.ceil(1+np.log2(len(train))))
    plt.hist(train[col], bins= k)
    plt.title(col)
    plt.xlabel("Valores")
    plt.ylabel("Frecuencia")
    plt.show()
    

#### Lectura del dataset

In [3]:
df_model=pd.read_csv('Modelar_UH2019.txt', sep="|" )
df_esti=pd.read_csv('Estimar_UH2019.txt', sep="|")

FileNotFoundError: ignored

#### Combinación de ambos datasets en uno único

    Concatenamos ambos datasets, es decir, el de modelar y el de estimar en un conjunto único. De esta forma, tenemos un mayor número de datos a los que manejar.

In [0]:
#axis=0 (eje horizontal) y #axis=1 (eje vertical)
#df_total= pd.concat([df_model, df_esti],axis=0) 
#df_total.shape
#df_total.tail()

# POST-ANÁLISIS 

#### Observar la distribución que siguen los datos

In [0]:
df_model.describe()

# PRE-PROCESSING

 https://scikit-learn.org/stable/modules/preprocessing.html

#### Crear un subconjunto donde no existen las categorías redundantes

In [0]:
noDesired_columns= ["HY_id","HY_provincia","HY_descripcion","HY_distribucion"]

def no_desired_columns(df,noDesired_columns ):
  total_columns= df.columns.values.tolist()

  sublist= [i for i in total_columns  if i not in noDesired_columns] # selecciona las columnas que no son las indicadas en noDesired_columns
  res_model = df[sublist]
  return res_model

s_df_model=no_desired_columns(df_model,noDesired_columns)

In [0]:
def desired_columns(df,desired_columns):
  total_columns= df.columns.values.tolist()

  sublist= [i for i in total_columns  if i in desired_columns] # selecciona las columnas que son las indicadas en Desired_columns
  res_model = df[sublist]
  return res_model

#### Comprobar si hay duplicados

In [0]:
# df_duplicated= no_desired_columns(s_df_model,["TARGET"])
# df_duplicated.loc[df_duplicated.duplicated,:]

In [0]:
# df_duplicated.duplicated().sum()

In [0]:
# s_df_model=df_duplicated.drop_duplicates(keep=False)

#### Transformamos las categorias nominales a númericas

De esta forma se podra utilizar a posteriori los arboles de decisión en la clasificación (https://pbpython.com/categorical-encoding.html)

Obervamos las categorias de tipo nominal o object

In [0]:
obj_df= s_df_model.select_dtypes(include=['object']).columns.tolist()
obj_df
#En el siguiente paso de missins eliminamos HY_cert_energ debido a la gran cantidad de nulls

##### Tipo

In [0]:
s_df_model["HY_tipo"].value_counts()

Label Encoding



*   Pasamos a categoria los valores nominales(HY_tipo y HY_cert_energ)






In [0]:
lb_make=preprocessing.LabelEncoder()
s_df_model['HY_tipo']=lb_make.fit_transform(s_df_model["HY_tipo"])

In [0]:
s_df_model["HY_tipo"].value_counts()

##### Scaling with outlayers

In [0]:
# robust_scaler= RobustScaler()
# robust_scaled_df = robust_scaler.fit_transform(s_df_model['HY_metros_utiles'])
# s_df_model['HY_metros_utiles_scale'] = pd.DataFrame(robust_scaled_df, columns=s_df_model['HY_metros_utiles'])

### Missing values 


https://towardsdatascience.com/the-tale-of-missing-values-in-python-c96beb0e8a9d
https://scikit-learn.org/stable/modules/impute.html
https://scikit-learn.org/stable/modules/impute.html


##### Cantidad de null y eliminación de columns

In [0]:
s_df_model.isna().sum()

Eliminamos el certificado de energia debido a la gran cantidad de nulls

In [0]:
s_df_model=s_df_model.drop(['HY_cert_energ'],axis=1)

#### Predición y relleno de nulls (https://stackoverflow.com/questions/40996051/predict-na-missing-values-with-machine-learning)

Nos quedamos con el dataframe sin nulls

In [0]:

nulls_columns=s_df_model.columns[s_df_model.isna().any()]

for i in nulls_columns: 
  target_column=i
  index_of_target = s_df_model.columns.get_loc(target_column)
  #Separamos en notNans y nans
  notnans= s_df_model.notnull().all(axis=1)
  df_notnans= s_df_model[notnans]
  df_nans= s_df_model.loc[~notnans]
  df_nans=no_desired_columns(df_nans,target_column)

  #Rellenamos los nans con la media
  df_nans=df_nans.fillna(0)

  #Objetivo y predcitores para el clasificador
  predictors=no_desired_columns(df_notnans,target_column)
  target=df_notnans[target_column]

  #Clasificador
  reg= DecisionTreeRegressor(min_samples_split=10,min_samples_leaf=1,random_state=None)
#   reg= RandomForestRegressor(n_jobs=2,n_estimators=500)
#   reg= GradientBoostingRegressor(n_estimators=500, max_depth=4,min_samples_split=2,learning_rate=0.01)
  reg.fit(predictors,target)


  #Predicción
  to_predict= df_nans
  name_pred_column=target_column
  to_predict[name_pred_column]= reg.predict(to_predict)
  


In [0]:
#Concatenamos el df predecido y el original sin nans
s_df_model=pd.concat([df_notnans, to_predict],axis=0) 

In [0]:
#s_df_model['TARGET']=s_df_model['TARGET'].fillna(-1)#en ele caso de que cojamos la parte de estimación también

##### Comprobación

In [0]:
np.isnan(s_df_model.any()).sum() 

In [0]:
np.isinf(s_df_model.all()).sum()

https://scikit-learn.org/stable/auto_examples/compose/plot_transformed_target.html#sphx-glr-auto-examples-compose-plot-transformed-target-py

In [0]:
# rng = np.random.RandomState(0)
# s_df_model[""]= np.sort(rng.normal(loc=0.5, scale=0.25, size=(25, 1)), axis=0)
# qt = QuantileTransformer(n_quantiles=10, random_state=0,output_distribution='normal')
# qt.fit_transform(s_df_model) 

### Normalización

https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py

##### Outlayers

In [0]:
s_df_model.columns

In [0]:
columns=['HY_metros_utiles','HY_metros_totales','GA_page_views','GA_quincena_ult','HY_precio','HY_precio_anterior',
          'IDEA_area','IDEA_densidad','IDEA_price_sale_residential','IDEA_price_rent_residential']
# columns=['HY_metros_utiles','HY_metros_totales','GA_page_views']

In [0]:
scaler = preprocessing.RobustScaler()
# scaler = preprocessing.MinMaxScaler()

s_df_model[columns] = scaler.fit_transform(s_df_model[columns])

In [0]:
uniform_distr = preprocessing.QuantileTransformer(output_distribution='normal', random_state=0)
s_df_model[columns] = uniform_distr.fit_transform(s_df_model[columns])

In [0]:
scalerNorm = preprocessing.Normalizer()
s_df_model[columns] = scalerNorm.fit_transform(s_df_model[columns])

In [0]:
# histogramas(s_df_model[columns])

##### To normal

In [0]:
s_df_model.columns

In [0]:
columns=['HY_metros_utiles','HY_metros_totales','GA_page_views','GA_quincena_ult','HY_precio','HY_precio_anterior',
          'IDEA_area','IDEA_densidad','IDEA_price_sale_residential','IDEA_price_rent_residential']
# columns=['HY_metros_utiles','HY_metros_totales','GA_page_views']

In [0]:
# scaler = preprocessing.RobustScaler()
# scaler = preprocessing.MinMaxScaler()

# s_df_model[columns] = scaler.fit_transform(s_df_model[columns])

In [0]:
uniform_distr = preprocessing.QuantileTransformer(output_distribution='normal', random_state=0)
s_df_model[columns] = uniform_distr.fit_transform(s_df_model[columns])

In [0]:
scalerNorm = preprocessing.Normalizer()
s_df_model[columns] = scalerNorm.fit_transform(s_df_model[columns])

In [0]:
# histogramas(s_df_model[columns])

##### Uniform Distribution

In [0]:
desired_columns= no_desired_columns(s_df_model,columns)

In [0]:
desired_columns.columns

In [0]:
uni_columns=['GA_exit_rate','GA_mean_bounce','GA_quincena_ini','HY_cod_postal','IDEA_demand_sale_residential',
             'IDEA_ind_alimentacion', 'IDEA_ind_elasticidad', 'IDEA_ind_liquidez',
             'IDEA_ind_riqueza', 'IDEA_ind_tienda', 'IDEA_ind_turismo',
             'IDEA_pc_1960', 'IDEA_pc_1960_69', 'IDEA_pc_1970_79', 'IDEA_pc_1980_89',
             'IDEA_pc_1990_99', 'IDEA_pc_2000_10', 'IDEA_pc_comercio',
             'IDEA_pc_industria', 'IDEA_pc_oficina', 'IDEA_pc_otros','IDEA_poblacion',
             'IDEA_rent_alquiler', 'IDEA_stock_rent_residential',
             'IDEA_stock_sale_residential', 'IDEA_unitprice_rent_residential',
             'IDEA_unitprice_sale_residential']

In [0]:
uniform_distr = preprocessing.QuantileTransformer(output_distribution='normal', random_state=0)
s_df_model[uni_columns] = uniform_distr.fit_transform(s_df_model[uni_columns])

In [0]:
scalerNorm = preprocessing.Normalizer()
s_df_model[uni_columns] = scalerNorm.fit_transform(s_df_model[uni_columns])

##### Target

Realizamos el ln del Target para acercarnos a  la distribución normal


In [0]:
s_df_model['TARGET']=np.log1p(s_df_model['TARGET'])

##### Category Encoding (http://contrib.scikit-learn.org/categorical-encoding/helmert.html, https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing)

In [0]:
no_desired=['HY_metros_utiles','HY_metros_totales','GA_page_views','GA_quincena_ult','HY_precio','HY_precio_anterior',
          'IDEA_area','IDEA_densidad','IDEA_price_sale_residential','IDEA_price_rent_residential',
            'GA_exit_rate','GA_mean_bounce','GA_quincena_ini','HY_cod_postal','IDEA_demand_sale_residential',
             'IDEA_ind_alimentacion', 'IDEA_ind_elasticidad', 'IDEA_ind_liquidez',
             'IDEA_ind_riqueza', 'IDEA_ind_tienda', 'IDEA_ind_turismo',
             'IDEA_pc_1960', 'IDEA_pc_1960_69', 'IDEA_pc_1970_79', 'IDEA_pc_1980_89',
             'IDEA_pc_1990_99', 'IDEA_pc_2000_10', 'IDEA_pc_comercio',
             'IDEA_pc_industria', 'IDEA_pc_oficina', 'IDEA_pc_otros','IDEA_poblacion',
             'IDEA_rent_alquiler', 'IDEA_stock_rent_residential',
             'IDEA_stock_sale_residential', 'IDEA_unitprice_rent_residential',
             'IDEA_unitprice_sale_residential']
category_features=no_desired_columns(s_df_model,no_desired)

In [0]:
category_features.columns

In [0]:
category_features=['HY_antiguedad', 'HY_ascensor', 'HY_num_banos', 'HY_num_garajes','HY_num_terrazas', 'HY_tipo', 'HY_trastero','IDEA_demand_rent_residential', 'IDEA_pc_residencial','IDEA_pc_trast_parking']

In [0]:
#df_model["HY_ascensor"].value_counts()

In [0]:
# s_df_model[category_features]=preprocessing.add_dummy_feature(s_df_model[category_features])
# enc = preprocessing.OrdinalEncoder()
# enc=preprocessing.OneHotEncoder(handle_unknown='ignore')
# enc.fit(s_df_model[category_features])
# s_df_model[category_features]=enc.transform(s_df_model[category_features])

In [0]:
# le=preprocessing.LabelEncoder()
# le.fit(s_df_model['HY_ascensor'])
# s_df_model['HY_ascensor']=le.transform(s_df_model['HY_ascensor'])

In [0]:
s_df_model[['HY_si_ascensor','HY_no_ascensor']]=pd.get_dummies(s_df_model['HY_ascensor'])


In [0]:
s_df_model=s_df_model.drop(['HY_ascensor'], axis=1)
s_df_model.columns

##### OneHotEncoder

In [0]:
# enc=OneHotEncoder(handle_unknown='ignore')
# enc.fit_transform(s_df_model)

##### Norm

In [0]:
# scalerNorm = preprocessing.Normalizer()
# s_df_model = scalerNorm.fit_transform(s_df_model)

#EXTRACCIÓN DE CARACTERÍSTICAS


Número de Fotografías

### Descripción 

# PRE-ANALISIS

https://www.machinelearningplus.com/plots/top-50-matplotlib-visualizations-the-master-plots-python/

In [0]:
s_df_model.head()

### Distribución

In [0]:
s_df_model.describe()

### Correlación

In [0]:
def plot_corr(df,size=10):

  corr = df.corr()
  fig, ax = plt.subplots(figsize=(size, size))
  cax = ax.matshow(corr,cmap='coolwarm', vmin=-1, vmax=1)
  fig.colorbar(cax)
  ticks = np.arange(0,len(df.columns),1)
  ax.set_xticks(ticks)
  plt.xticks(rotation=90)
  ax.set_yticks(ticks)
  ax.set_xticklabels(df.columns)
  ax.set_yticklabels(df.columns)
  plt.show()

In [0]:
 plot_corr(s_df_model,18)

### Histogramas

In [0]:
#  histogramas(s_df_model)

### Boxplot

In [0]:
def boxplots(train):  
  for col in train:
    plt.boxplot(train[col], dict(markerfacecolor='g', marker='D'),showfliers=True)
    plt.title(col)
    plt.xlabel("Valores")
    plt.ylabel("Frecuencia")
    plt.show()

In [0]:
#boxplots(s_df_model)

# CLASIFICACIÓN

#### Obtenemos las categorias predictoras por un lado  y el target por otro

#### Separamos el conjunto en un conjunto de Training y un conjunto de Test

In [0]:
s_df_model["is_train"]= np.random.uniform(0,1,len(s_df_model))<=0.75

Partimos el conjunto en dos subconjuntos en función del valor -1 indicando cual era el training y el estimador

In [0]:
#s_df_model["is_train"]= s_df_model['TARGET'].values!=-1

Vemos en el histograma como se reparte los datos entre Train y test

In [0]:
plt.hist(s_df_model.is_train)

Partimos en test y train

In [0]:
train, test = s_df_model[s_df_model["is_train"]==True],s_df_model[s_df_model["is_train"]==False]
train.drop("is_train",axis=1,inplace=True)
test.drop("is_train",axis=1,inplace=True)

In [0]:
predictors_train= no_desired_columns(train,'TARGET') 
predictors_test= no_desired_columns(test,'TARGET') 

target= train['TARGET']# hay que tener en cuenta que se empieza en el cero

### Regression Tree 

Creamos el módelo

In [0]:
# regtree= DecisionTreeRegressor(min_samples_split=5,min_samples_leaf=1,random_state=None)
# regtree.fit(predictors,target)

#### Prediction

In [0]:
#res=test
# res["regtree_preds"]= regtree.predict(predictors_test)

#train[["regtree_preds","TARGET"]]

Median absolute error

In [0]:

# mae_regtree=mean_absolute_error(np.expm1(test["TARGET"]),np.expm1(res["regtree_preds"]))    
# mae_regtree=mean_absolute_error(to_predict["TARGET"],to_predict["regtree_preds"])                    

### Regression Random Forest
    

*   n_jobs-> número de tareas en paralelo
*   obb_score-> true(aleatorio)
*   n_estimators.-> núm. de estimadores que se va a tener



In [0]:
# forest= RandomForestRegressor(n_jobs=2,n_estimators=500)
# forest.fit(predictors,target)

#### prediction

In [0]:
#res=test
# res["rforest_preds"]=forest.predict(predictors_test)

In [0]:
# mae_rforest=mean_absolute_error(np.expm1(test["TARGET"]),np.expm1(res["rforest_preds"]))

### Gradient Boosting Regressor

In [0]:
gradient= GradientBoostingRegressor(n_estimators=500, max_depth=5,min_samples_split=2,learning_rate=0.01)
gradient.fit(predictors_train,target)

#### Prediction

In [0]:
res=test
res["gboost_preds"]=gradient.predict(predictors_test)

In [0]:
mae_gboost=mean_absolute_error(np.expm1(test["TARGET"]),np.expm1(res["gboost_preds"]))
# mae_gboost=mean_absolute_error(to_predict["TARGET"],to_predict["gboost_preds"])

# RESULTADO (SCORE)

#### MAE de los clasificadores

In [0]:
# df_res_classifiers = pd.DataFrame({'mae_rforest': [mae_rforest], 'mae_regtree': [mae_regtree],'mae_gboost':[mae_gboost]})
# df_res_classifiers = pd.DataFrame({'mae_regtree': [mae_regtree],'mae_gboost':[mae_gboost]})
mae_gboost
# df_res_classifiers

#### Fichero final

In [0]:
# test["HY_id"]=df_model["HY_id"]
# test= test[["HY_id","regtree_preds"]]
# test= test.rename(columns={"regtree_preds":"TM_Est"})
# test.to_csv(r'Minsait_UJI_Math.random()_1.txt', header=True, index=None, sep='|', mode='w')
# df=pd.read_csv('Minsait_UJI_Math.random()_1.txt', sep="|")

In [0]:
# df.head()

# df.shape